# Projeto Supercomputação - TSP

O objetivo deste projeto será comparar 2 diferentes implementações paralelas de busca local para o problema do Travel Salesman Problem: 
* Paralelismo multi-core
* Paralelismo em GPU 

OBS: Para deixar o algoritmo mais rápido, as antigas implementações foram alteradas para realizar a troca comparando apenas com o primeiro índice

## Setup

### Compilando os códigos

In [8]:
import subprocess

# Compile C++ code for global search
#subprocess.call(["g++", "-std=c++11", "-o", "busca-exaustiva", "busca-exaustiva/main.cpp"])

# Compile C++ busca local normal code
subprocess.call(["g++", "-std=c++11", "-o", "busca-local-sequencial", "busca-local-sequencial.cpp"])

# Compile C++ busca local com openmp
subprocess.call(["g++", "-std=c++11", "-fopenmp", "-o", "busca-local-cpu", "busca-local-cpu.cpp"])

# Compile C++ busca local com CUDA
#subprocess.call(["nvcc", "-arch=sm_70", "-std=c++14", "-o", "busca-local-gpu", "busca-local-gpu.cu"])

clang: error: unsupported option '-fopenmp'
clang: error: unsupported option '-fopenmp'


1

In [3]:
import time
from subprocess import Popen, PIPE
def run(exec, entrada):
    start = time.perf_counter()
    command = "./" + exec + " < " + entrada
    process = Popen(command, stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
    stdout, stderr = process.communicate()
    end = time.perf_counter()
    return end - start

def run_all(exec, entradas):
    times = []
    for entrada in entradas:
        times.append(run(exec, entrada))
    return times

entradas = ["entradas/in-{}.txt".format(i) for i in range(150)]

In [9]:
# busca local sequencial
times_busca_local_sequencial = run_all("busca-local-sequencial", entradas)

In [32]:
# busca local com openmp
times_busca_local_cpu = run_all("busca-local-cpu", entradas)

In [33]:
# busca local com CUDA
times_busca_local_gpu = run_all("busca-local-gpu", entradas)


In [14]:
import matplotlib.pyplot as plt
import numpy as np

times = [times_busca_local_sequencial, times_busca_local_cpu, times_busca_local_gpu]

# plot all in one figure
def plot_all(times, title):
    plt.figure(figsize=(10, 5))
    plt.title(title)
    plt.xlabel("Entradas")
    plt.ylabel("Tempo (s)")
    plt.plot(times[0], label="Busca Local Sequencial")
    plt.plot(times[1], label="Busca Local CPU")
    plt.plot(times[2], label="Busca Local GPU")
    plt.legend()
    plt.show()

plot_all(times, "Busca Local")


ModuleNotFoundError: No module named 'matplotlib'

## Comparando o desempenho das implementações

### Se você pudesse escolher um método para resolver este problema, qual seria?

O método escolhido dependeria do tamanho da entrada. Observa-se no gráfico acima que para entradas maiores, a GPU se sai melhor. Já para entradas menores, a melhor escolha seria a implementação paralelizada da busca local, uma vez que obteve tempos bem menores.


### Valeria a pena gastar dinheiro comprando uma CPU com mais cores ou uma GPU potente?


Depende das necessidades do usuário. Se ele necessita lidar com pouco volume de dados, a melhor opção seria uma implementação paralelizada na própria CPU. Caso ele lide com um volume de dados alto, a melhor escolha, sem dúvidas é realizar a implementação na GPU.

### Vale a pena esperar pelo resultado da busca exaustiva?

Sim, na busca exaustiva são feitas inúmeras operações idênticas para encontrar todos as possíveis combinações. Dito isso, a implementação paralelizada em gpu traria bons resultados em entradas maiores, ao contrário da CPU que tem sua capacidade de processar dados levada ao limite, tornando a implementação lenta e pouco eficiente para entradas maiores.